In [24]:
import pandas as pd
import numpy as np
import math

In [25]:
HISTORY = 6 #indicates how many past datapoints are considered (e.g prediction based on pas 6 hours)
FUTURE = 1  #indicates prediction into future (e.g 1 timestep into the future)
OVERLAP = 1 #indicates how much the data is overlapped

In [26]:
# data = pd.read_csv("../data/finaldata/1h_allData_filtered.csv")
data = pd.read_csv("../data/finaldata/1h_allData_filtered.csv")

In [27]:
data.head()

,time,date,weekday,hour,loc,temp_pi,temp_sbs,temp_blue,hum_pi,hum_sbs,...,wind,wind_speed,wind_direction,cloud_cover,pressure,AQI,sunrise,noon,sunset,daylight
0,2021-03-18 17:00:00+01:00,2021-03-18,3,17,A,25.128571,NaN,4.886313,36.428571,NaN,...,NaN,11.119281,60.945396,62.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719
1,2021-03-18 18:00:00+01:00,2021-03-18,3,18,A,24.350000,NaN,4.706314,34.175000,NaN,...,NaN,9.826088,61.557068,100.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719
2,2021-03-18 19:00:00+01:00,2021-03-18,3,19,A,6.480000,NaN,4.456314,49.986667,NaN,...,NaN,9.178235,64.440025,100.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719
3,2021-03-18 20:00:00+01:00,2021-03-18,3,20,A,3.066667,NaN,4.036314,61.100000,NaN,...,NaN,7.235910,84.289406,100.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719
4,2021-03-18 21:00:00+01:00,2021-03-18,3,21,A,2.458333,NaN,3.356314,66.141667,NaN,...,NaN,6.638072,139.398700,100.0,NaN,NaN,2021-03-18 06:32:56.209197+01:00,2021-03-18 12:33:56+01:00,2021-03-18 18:35:30.551916+01:00,0 days 12:02:34.342719


In [28]:
hours = pd.to_datetime(data['time'], utc=True).dt.hour

In [29]:
locs = data['loc']

In [30]:
features = ['temp_pi', 'hum_pi', 'PM1_pi', 'PM2.5_pi', 'PM10_pi']

In [31]:
targets = ['PM2.5_pi', 'PM10_pi']

In [32]:
data = data[features]

In [33]:
data

,temp_pi,hum_pi,PM1_pi,PM2.5_pi,PM10_pi
0,25.128571,36.428571,4.015714,9.323571,14.962143
1,24.350000,34.175000,2.905000,8.352500,14.282500
2,6.480000,49.986667,5.054000,11.749333,18.859333
3,3.066667,61.100000,7.343333,17.540833,28.408333
4,2.458333,66.141667,6.643333,14.654167,23.088333
...,...,...,...,...,...
1075,18.481818,37.609091,2.420909,4.647273,6.926364
1076,16.550000,50.075000,3.684167,8.526667,13.664167
1077,14.775000,64.150000,6.013333,12.902500,20.120833
1078,14.375000,64.841667,3.553333,6.644167,9.780833


In [34]:
features_new = []
for i in range(HISTORY):
    features_new += [f + "_t" + str(i) for f in features]

In [35]:
features_new += targets 
features_new += ['hour', 'loc']

In [36]:
x = math.ceil(data.shape[0]/OVERLAP)
y = len(features_new) 
data_new = np.zeros((x, y))

In [37]:
k = 0
for i in range(HISTORY - 1, data.shape[0] - 1, OVERLAP):
    temp = np.zeros(len(features_new))
    for j in range(HISTORY):
        temp[j*(len(features)):(j+1)*(len(features))] = data.iloc[i - j, :]
    
    temp[len(features)*HISTORY] = data.iloc[i+1, 3]
    temp[len(features)*HISTORY+1] = data.iloc[i+1, 4]
    temp[len(features)*HISTORY+2] = hours[i]
    temp[len(features)*HISTORY+3] = 0 if locs[i] == 'A' else 1
    data_new[k,:] = temp
    k += 1

In [38]:
data_new = pd.DataFrame(data_new, columns=features_new)

In [39]:
data_new

,temp_pi_t0,hum_pi_t0,PM1_pi_t0,PM2.5_pi_t0,PM10_pi_t0,temp_pi_t1,hum_pi_t1,PM1_pi_t1,PM2.5_pi_t1,PM10_pi_t1,...,PM10_pi_t4,temp_pi_t5,hum_pi_t5,PM1_pi_t5,PM2.5_pi_t5,PM10_pi_t5,PM2.5_pi,PM10_pi,hour,loc
0,2.200000,67.791667,7.280000,14.491667,21.935833,2.458333,66.141667,6.643333,14.654167,23.088333,...,14.282500,25.128571,36.428571,4.015714,9.323571,14.962143,11.823333,17.340833,21.0,0.0
1,2.075000,69.941667,6.376667,11.823333,17.340833,2.200000,67.791667,7.280000,14.491667,21.935833,...,18.859333,24.350000,34.175000,2.905000,8.352500,14.282500,16.170000,24.518333,22.0,0.0
2,2.050000,71.791667,8.087500,16.170000,24.518333,2.075000,69.941667,6.376667,11.823333,17.340833,...,28.408333,6.480000,49.986667,5.054000,11.749333,18.859333,13.952500,21.710833,23.0,0.0
3,1.983333,69.816667,6.544167,13.952500,21.710833,2.050000,71.791667,8.087500,16.170000,24.518333,...,23.088333,3.066667,61.100000,7.343333,17.540833,28.408333,10.430833,15.020833,0.0,0.0
4,1.758333,69.583333,5.843333,10.430833,15.020833,1.983333,69.816667,6.544167,13.952500,21.710833,...,21.935833,2.458333,66.141667,6.643333,14.654167,23.088333,7.693333,11.190000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1076,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1077,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
1078,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0


In [22]:
data_new.to_csv('30min_model_data_pi.csv')